In [161]:
#image analysis
import skimage.io
import imageio
import alienlab.plot
from alienlab.improcessing import normalize, grey_to_rgb, make_binary
import alienlab.segment
from alienlab.fo import FramesOperator
import alienlab.io
from scipy import optimize
import glob
from alienlab.regression_func import *
import copy
from VoltageIntensityClass import VoltageIntensity
from tqdm import tqdm

from joblib.externals.loky import set_loky_pickler
from joblib import parallel_backend
from joblib import Parallel, delayed
from joblib import wrap_non_picklable_objects
from skimage.transform import resize

import tifffile as tiff

#interactive widget packages
from ipywidgets import interact, interactive, fixed, interact_manual
from tkinter.filedialog import askopenfilename, askdirectory

from VoltageIntensityClass import VoltageIntensity

import time
import os
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd

%matplotlib ipympl
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as wdg  # Using the ipython notebook widgets

#%matplotlib inline


def residuals(parameters,x_data,y_observed,func):
    '''
    Compute residuals of y_predicted - y_observed
    where:
    y_predicted = func(parameters,x_data)
    '''
    return func(parameters,x_data) - y_observed

def clip(input_image, high = 95, low = 5):
    im = copy.copy(input_image)
    m = np.median(im)
    im[im<np.percentile(im, low)]=np.percentile(im, low)
    im[im>np.percentile(im, high)]=np.percentile(im, high)
    return im

def platt(parameters, xdata):
    M = parameters[0]
    alpha = parameters[1]
    return M*(1- np.exp(-alpha*xdata/M))

def exp_decay(parameters, xdata):
    '''
    Calculate an exponetial decay of the form:
    S= a * exp(-xdata/b)
    '''
    A = parameters[0]
    tau = parameters[1]
    y0 = parameters[2]
    return A * np.exp(-xdata/tau) + y0

def modele_direct(parameters, x_data):
    a = parameters[0]
    b = parameters[1]
    c = parameters[2]
    d = parameters[3]
    e = parameters[4]
    return (a*x_data +b*np.sqrt(c*I**2+d*I+1)+e)/I

def modele_inverse(parameters, x_data):
    a = parameters[0]
    b = parameters[1]
    c = parameters[2]
    d = parameters[3]
    e = parameters[4]
    f = parameters[5]
    return a*I + b*np.sqrt(c*I**2+d*I+1) + e

In [162]:
#file_path = "ENTER NAME OF THE FILE YOU DROPPED IN THE FILE SECTION HERE"
file_folder = askdirectory(title = 'Select an experiment folder') # pops up a window to select your file
# uncomment this line if you use this jupyter notebook locally
#'G:/DREAM/from_github/PAMFluo/Experiments/2021-06-24_12_12_Ek_video'


In [163]:
show = True #option to output intermediary images in the segmentation process

# Import video file in HQ and select ROI
file_path = file_folder + "/video_1.tiff"
direc = os.path.split(file_path)[0]

# Initialize plotting tools
g = alienlab.plot.ShowFigure()
g.figsize = (15,7)
g.save_folder = "images"
g.date = False
p = alienlab.plot.PlotFigure()
p.figsize = (15,7)
p.save_folder = "images"
p.date = False

In [164]:
# read the stacked frame. dim = NxHxW (N images in the video, Heigt, Width)

frames_full = skimage.io.imread(file_path)

#frames_full = np.stack([frames_full[:,:,1]]*10, 0) 
#uncomment this line if you have a single RGB image. The [:,:,1] stands for selection of the green channel

FO = FramesOperator(frames_full)
im = normalize(FO.frames[0], 0, 1)
im = grey_to_rgb(im)*255

# CROP
#y, x = alienlab.io.select_roi(np.uint8(im)) #select area of interest
#FO.x = x
#FO.y = y
#FO.crop() #crop image

start_time = time.time()
FO.compute_stats() #compute various statistical values on the frames and the pixels
FO.normalize(0, 1)
print("--- Computed frames statistics in %04f seconds ---" % (time.time() - start_time))

#FO.global_stats: each array has size N, number of frames and represents the stats of each frame
#FO.frames_stats: each array has size FO.x, FO.y and is an image representing the N frames stats overlayed

if show:
    p.title = 'statistics'
    p.xlabel = 'frame number'
    p.ylabel = 'amplitude'
    p.label_list = ['max', 'min', 'mean', 'std']
    fig = p.plotting(np.asarray(FO.inds), [FO.global_stats['max'], 
                        FO.global_stats['min'], 
                        FO.global_stats['mean']])
    p.save_name = 'frames_stats'
    p.saving(fig)

''' IMAGE SEGMENTATION '''

# selection of the frames with high dynamics that will be used for the image segmentation process.
# Let M be the highest value taken by a pixel in all the frames of the video. The frame F is kept for processing only if at
# least one pixel in the frame F has a value above 0.8*M. 
FO.selected_inds = FO.select_frames(FO.global_stats['max'], FO.global_stats['max'].max()*0.8)


--- Computed frames statistics in 0.351281 seconds ---


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [165]:

plt.figure(figsize = (5, 5))
FO.selected_inds = FO.select_frames(FO.global_stats['max'], FO.global_stats['max'].max()*0.98) # Select only images with high intensity to increase contrast and lower computation time

plt.imshow(FO.frames[FO.selected_inds].sum(axis = 0), cmap = 'gray')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [166]:
import copy
L,H,_ = im.shape
mask = copy.copy(im[:,:,0])*0
d = 0
N  = 12
for i in range(L//N):
    for j in range(H//N):
        mask[i*N:(i+1)*N,j*N:(j+1)*N] = d
        d+=1
plt.figure()
plt.imshow(mask)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [167]:
# Collect item labels

# Item time trajectories with overlaps
# create a dictionnary with one entry for each item:
'''
{ '1.0': {'x_coords': np array, x coordinates in HQ}
            'y_coords': np array,  y coordinates in HQ
            'binned_coords': set, couples of (x,y) coordinates in binned video
            'surface': number of pixels in the item in HQ
            'pixel_values': array, size: (N, s) where N is number of frames and s surface
            'mean': array, size N, mean value of the item intensity for each frame
            'std':  array, size N, std value of the item intensity for each frame
            'remains' : True, the item is present in this segmentation step
            }
'2.0': {'x_coords'...
                }
    }
'''
segmented = mask
items = np.unique(segmented) #returns the set of values in items, corresponds to the values of the markers of local_maxima

items_dict = {}
for k in tqdm(items):
    key = str(k)
    items_dict[key] = {}
    x_coords, y_coords = np.nonzero(segmented == k)
    items_dict[key]['x_coords'] = x_coords
    items_dict[key]['y_coords'] = y_coords
    pixel_values = FO.frames[:,x_coords, y_coords]
    items_dict[key]['pixel_values'] = pixel_values
    items_dict[key]['surface'] = pixel_values.shape[1]
    items_dict[key]['mean'] = np.mean(pixel_values, axis = 1)
    items_dict[key]['std'] = np.std(pixel_values, axis = 1)
    items_dict[key]['remains'] = True




100%|█████████████████████████████████████████████████████████████████████████████| 4000/4000 [00:05<00:00, 690.72it/s]


In [168]:
data_sequence = {}
video_sequence = {}
fig, axs = plt.subplots(2, 2, figsize=(10, 8))
for i in range(3):

    file_path = glob.glob(file_folder + "/*_ojip_curve_%d.csv"%i)[0] # pops up a window to select your file
    data_sequence[i] = pd.read_csv(file_path)
    data_sequence[i].plot( x = "time (s) o", y = "voltage (V) o", ax= axs[0][i%2])

    file_path = file_folder + "/video_%d.tiff"%i


    video = tiff.imread(file_path)
    video_sequence[i]=video
    axs[1][i%2].plot(np.mean(video, axis = (1,2)))
        
plt.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[]

In [169]:

NPQ_tot = []
video_algae = []
algae_list = list(items_dict.keys())
result_npq = copy.copy(mask)*0
for P in [5]:#[len(algae_list)]:#[8, 10, 12, 15, 18, 20, 25, 30, 35, 40, 60, 90, 120, 180, 200, 250, 300, 400, 500, 600, 700]:

    #for algae in random.sample(algae_list, P):
    for algae in tqdm(algae_list):
        #if int(algae):# in ['1', '80', '400', '250']:
            i = 0


            x_coords =  items_dict[algae]['x_coords']
            y_coords =  items_dict[algae]['y_coords']
            
            exposed = np.sum(video_sequence[2][0:5, x_coords, y_coords])
            dark =  np.sum(video_sequence[0][0:5, x_coords, y_coords])
            trace = np.sum(video_sequence[1][:, x_coords, y_coords], axis = (1))
            npq = (dark-exposed)/dark
            video_algae.append(np.array(trace))
            NPQ_tot.append(npq)
          
            result_npq[mask==int(float(algae))] = npq

result_npq[result_npq != result_npq] = 1
plt.figure()
plt.imshow(clip(result_npq, high = 80, low = 10))
plt.figure()
plt.imshow(FO.frames[FO.selected_inds].sum(axis = 0), cmap = 'gray')


  0%|                                                                                         | 0/4000 [00:00<?, ?it/s]<ipython-input-169-7a4c2ac8b1f1>:19: RuntimeWarning: overflow encountered in ulong_scalars
  npq = (dark-exposed)/dark
100%|████████████████████████████████████████████████████████████████████████████| 4000/4000 [00:01<00:00, 3354.40it/s]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [170]:
plt.figure()
plt.hist(clip(result_npq,high = 90, low = 10).flatten())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([70704., 24352., 18000., 23040., 39456., 61920., 83088., 99216.,
        68976., 99792.]),
 array([0.23308271, 0.26325315, 0.29342359, 0.32359403, 0.35376447,
        0.38393492, 0.41410536, 0.4442758 , 0.47444624, 0.50461668,
        0.53478712]),
 <BarContainer object of 10 artists>)

In [174]:
x0 = [1.1, 8, 0.1]
def get_fit(decay, time):    

    parameters_estimated = optimize.least_squares(residuals,  x0, bounds = (0,1e8),
                                args = (time, decay, exp_decay))
    
    
    return np.array(parameters_estimated.x)


def make_fit(decay, time_array):
    plt.figure()
    params = get_fit(decay, time_array)
    plt.plot(time_array, exp_decay(params, time_array), label = params[1])
    plt.plot(time_array, decay, '.')
    plt.legend()
    
start = 5
decay = np.mean(video_sequence[1], axis = (1,2))[start:]
time_array = np.linspace(0, len(decay), len(decay))


#time_array = np.array(data_sequence[1]["time (s) o"].values)[30:-10]
#decay = np.array(data_sequence[1]["voltage (V) o"].values)[30:-10]

make_fit(decay, time_array)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [176]:
time_array = np.linspace(0, len(video_algae[i]), len(video_algae[i]))
params = Parallel(n_jobs = -1 )(delayed(get_fit)(video_algae[i][start:], time_array[start:]) for i in range(len(video_algae)))


In [177]:
params = np.array(params)
tau = params[:,1]
im_tau = tau.reshape((L//N,H//N))
plt.figure()
plt.imshow(clip(1/im_tau))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [178]:
plt.figure()
plt.hist(clip(tau, high = 95, low = 5))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([458., 484., 625., 626., 571., 412., 309., 162.,  94., 259.]),
 array([ 7.40189712,  7.67196095,  7.94202477,  8.2120886 ,  8.48215242,
         8.75221625,  9.02228007,  9.2923439 ,  9.56240772,  9.83247155,
        10.10253537]),
 <BarContainer object of 10 artists>)

In [179]:

# Create a random image
fig, axs = plt.subplots(1, 2, figsize=(10, 4))
axs[0].imshow(clip(im_tau))
axs[0].axis('off')

# Create and display textarea widget
txt = wdg.Textarea(
    value='',
    placeholder='',
    description='event:',
    disabled=False
)
display(txt)
image_algae = np.array(video_algae).reshape((L//N,H//N, len(video_algae[0])))
coords = []

# Define a callback function that will update the textarea
def onclick(event):
    global ix, iy
    ix, iy = event.xdata, event.ydata
    txt.value = str(event)#"x= %d, y = %d"%(ix, iy)

    global coords
    coords.append((ix, iy))

    #if len(coords) == 2:
    #    fig.canvas.mpl_disconnect(cid)
    """
    #txt.value = str(event.x)  # Dynamically update the text box above
    x = np.copy(int(event.x))
    y = np.copy(int(event.y))
    """
    decay = image_algae[int(iy), int(ix), 3:]
    time_array = np.linspace(0, len(decay), len(decay))

    make_fit(decay, time_array)
    params = get_fit(decay, time_array)
    axs[1].plot(time_array, exp_decay(params, time_array), label = params[1])
    axs[1].plot(time_array, decay, '.')
    axs[1].legend()
    plt.tight_layout()
# Create an hard reference to the callback not to be cleared by the garbage collector
ka = fig.canvas.mpl_connect('button_press_event', onclick)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Textarea(value='', description='event:', placeholder='')

In [112]:
#WAITTTTTTTTTTTTTTT

In [180]:
np.save("G:/DREAM/from_github/PAMFluo/Figures/NPQ/imtau_sick_5_09_16.npy", im_tau)
np.save("G:/DREAM/from_github/PAMFluo/Figures/NPQ/im_sick_5_09_16.npy", FO.frames[FO.selected_inds].sum(axis = 0))



# Compare

In [181]:
folder_list = glob.glob("G:/DREAM/from_github/PAMFluo/Experiments/*qE_OJIP")

data_sequence = {}
video_sequence = {}
fig, axs = plt.subplots(2, 2, figsize=(10, 8))
for file_folder in folder_list[5:]:
    print(file_folder)
    for i in range(3):
        
        file_path = glob.glob(file_folder + "/*_ojip_curve_%d.csv"%i)[0] # pops up a window to select your file
        data_sequence[i] = pd.read_csv(file_path)
        data_sequence[i].plot( x = "time (s) o", y = "voltage (V) o", ax= axs[0][i%2])

        file_path = file_folder + "/video_%d.tiff"%i


        video = tiff.imread(file_path)
        video_sequence[i]=video
        axs[1][i%2].plot(np.mean(video, axis = (1,2)))
        
plt.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

G:/DREAM/from_github/PAMFluo/Experiments\2021-09-14_17_33_qE_OJIP
G:/DREAM/from_github/PAMFluo/Experiments\2021-09-14_17_39_qE_OJIP
G:/DREAM/from_github/PAMFluo/Experiments\2021-09-15_11_23_qE_OJIP
G:/DREAM/from_github/PAMFluo/Experiments\2021-09-15_14_19_qE_OJIP
G:/DREAM/from_github/PAMFluo/Experiments\2021-09-15_14_24_qE_OJIP
G:/DREAM/from_github/PAMFluo/Experiments\2021-09-15_15_08_qE_OJIP
G:/DREAM/from_github/PAMFluo/Experiments\2021-09-15_16_46_qE_OJIP
G:/DREAM/from_github/PAMFluo/Experiments\2021-09-15_17_13_qE_OJIP
G:/DREAM/from_github/PAMFluo/Experiments\2021-09-15_18_11_qE_OJIP
G:/DREAM/from_github/PAMFluo/Experiments\2021-09-15_18_23_qE_OJIP
G:/DREAM/from_github/PAMFluo/Experiments\2021-09-16_14_16_qE_OJIP
G:/DREAM/from_github/PAMFluo/Experiments\2021-09-16_15_19_qE_OJIP
G:/DREAM/from_github/PAMFluo/Experiments\2021-09-16_15_37_qE_OJIP
G:/DREAM/from_github/PAMFluo/Experiments\2021-09-16_15_52_qE_OJIP
G:/DREAM/from_github/PAMFluo/Experiments\2021-09-16_16_00_qE_OJIP
G:/DREAM/f

[]

In [151]:

fig, axs = plt.subplots(2, 3, figsize=(14, 7))

vmin = 3
vmax = 5

healthy = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/imtau_health_1_09_16.npy")
axs[0][0].imshow(clip(healthy), vmin=vmin, vmax=vmax)
healthy_base = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/im_health_1_09_15.npy")
axs[1][0].imshow(clip(healthy_base), cmap = "gray")


healthy = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/imtau_health_2_09_16.npy")
axs[0][1].imshow(clip(healthy), vmin=vmin, vmax=vmax)
healthy_base = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/im_health_2_09_15.npy")
axs[1][1].imshow(clip(healthy_base), cmap = "gray")


healthy = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/imtau_health_3_09_16.npy")
imcb = axs[0][2].imshow(clip(healthy), vmin=vmin, vmax=vmax)
healthy_base = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/im_health_3_09_15.npy")
axs[1][2].imshow(clip(healthy_base), cmap = "gray")

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
fig.colorbar(imcb, cax=cbar_ax)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [182]:

fig, axs = plt.subplots(2, 3, figsize=(14, 7))

for axi in axs:
    for ax in axi:
        ax.axis("off")
    
vmin = 2
vmax = 9

healthy = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/imtau_sick_1_09_16.npy")
axs[0][0].imshow(clip(healthy), vmin=vmin, vmax=vmax)

healthy_base = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/imtau_health_1_09_16.npy")
axs[1][0].imshow(clip(healthy_base), vmin=vmin, vmax=vmax)


healthy = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/imtau_sick_2_09_16.npy")
axs[0][1].imshow(clip(healthy), vmin=vmin, vmax=vmax)
healthy_base = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/imtau_health_2_09_16.npy")
axs[1][1].imshow(clip(healthy_base), vmin=vmin, vmax=vmax)


healthy = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/imtau_sick_5_09_16.npy")
imcb = axs[0][2].imshow(clip(healthy), vmin=vmin, vmax=vmax)
healthy_base = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/imtau_health_3_09_16.npy")
axs[1][2].imshow(clip(healthy_base), vmin=vmin, vmax=vmax)

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
fig.colorbar(imcb, cax=cbar_ax)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:

fig, axs = plt.subplots(2, 3, figsize=(14, 7))

vmin = 1
vmax = 20

healthy = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/imtau_semihealthy_09_15.npy")
axs[0][0].imshow(clip(healthy), vmin=vmin, vmax=vmax)
healthy_base = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/im_semihealthy_09_15.npy")
axs[1][0].imshow(clip(healthy_base), cmap = "gray")

sick = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/imtau_semihealthy_bis_09_15.npy")
axs[0][1].imshow(clip(sick), vmin=vmin, vmax=vmax)
sick_base = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/im_semihealthy_bis_09_15.npy")
axs[1][1].imshow(clip(sick_base), cmap ="gray")


sick = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/imtau_semihealthy_ter_09_15.npy")
imcb = axs[0][2].imshow(clip(sick), vmin=vmin, vmax=vmax)
sick_base = np.load("G:/DREAM/from_github/PAMFluo/Figures/NPQ/im_semihealthy_ter_09_15.npy")
axs[1][2].imshow(clip(sick_base), cmap ="gray")

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
fig.colorbar(imcb, cax=cbar_ax)

<ipython-input-21-cf672a71c238>:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axs = plt.subplots(2, 3, figsize=(14, 7))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …